# Project Benson

## Exploring...

In [88]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
import seaborn as sns
from numpy import linalg

import math
import patsy

from statsmodels.formula.api import ols

%matplotlib inline

In [4]:
!python -V

Python 3.6.3 :: Anaconda custom (64-bit)


In [5]:
print("Pandas version:",pd.__version__)
print("Numpy version:",np.__version__)

Pandas version: 0.20.3
Numpy version: 1.13.3


### Pick a week and play...

In [6]:
df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_170415.txt')

In [7]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,00:00:00,REGULAR,6127245,2076158
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,04:00:00,REGULAR,6127285,2076165
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,08:00:00,REGULAR,6127304,2076196
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,12:00:00,REGULAR,6127409,2076284
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,16:00:00,REGULAR,6127654,2076350


The field description from the source website:

Field Description

C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS


C/A      = Control Area (A002)
UNIT     = Remote Unit for a station (R051)
SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)
STATION  = Represents the station name the device is located at
LINENAME = Represents all train lines that can be boarded at this station
           Normally lines are represented by one character.  LINENAME 456NQR repersents train server for 4, 5, 6, N, Q, and R trains.
DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND   
DATE     = Represents the date (MM-DD-YY)
TIME     = Represents the time (hh:mm:ss) for a scheduled audit event
DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. 
ENTRIES  = The comulative entry register value for a device
EXIST    = The cumulative exit register value for a device


So... things we need to do:

* Strip out anything but "REGULAR" audits
* Create derived columns
  * Entry hits since last REGULAR audit
  * Exit hits since last REGULAR audit
  * Total of the two above
* Determine how to aggregate the above per station
* Incorporate geographical data per station


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201197 entries, 0 to 201196
Data columns (total 11 columns):
C/A                                                                     201197 non-null object
UNIT                                                                    201197 non-null object
SCP                                                                     201197 non-null object
STATION                                                                 201197 non-null object
LINENAME                                                                201197 non-null object
DIVISION                                                                201197 non-null object
DATE                                                                    201197 non-null object
TIME                                                                    201197 non-null object
DESC                                                                    201197 non-null object
ENTRIES                           

Let's start small.

It will be easier here to focus on just one device actually, even moreso than one station.

So... what's going on with this data?  What's the real structure?  The field descriptions aren't quite so clear if you aren't familiar with the context...

In [23]:
df.apply(lambda x: x.value_counts().count())

C/A                                                                        732
UNIT                                                                       466
SCP                                                                        230
STATION                                                                    374
LINENAME                                                                   114
DIVISION                                                                     6
DATE                                                                         7
TIME                                                                     12844
DESC                                                                         2
ENTRIES                                                                 168614
EXITS                                                                   167922
dtype: int64

Hmm... these are interesting counts.  The highest spread of the identifiers suggests "C/A" is truly the column of interest here - our key, so to speak.

Found better guidance for the data:
* [Turnstile Data Overview](https://data.ny.gov/api/views/ekwu-khcy/files/30554bf0-2ec6-4804-a91b-85185a96a877?download=true&filename=MTA_Turnstile_Data_Overview.pdf)
* [Turnstile Data Dictionary](https://data.ny.gov/api/views/ekwu-khcy/files/e35d46bb-d988-44c5-a170-8736c1c773af?download=true&filename=MTA_Turnstile_Data_DataDictionary.pdf)


In [28]:
df.iloc[:,:2].drop_duplicates().count()

C/A     732
UNIT    732
dtype: int64

In [30]:
df.iloc[:,1:3].drop_duplicates().count()

UNIT    4691
SCP     4691
dtype: int64

In [32]:
df.loc[:,['UNIT','STATION']].drop_duplicates().count()

UNIT       482
STATION    482
dtype: int64

In [70]:
df.loc[:,['UNIT','STATION']].drop_duplicates().sort_values('STATION')

,UNIT,STATION
33027,R248,1 AV
162093,R180,103 ST
141872,R191,103 ST
48682,R314,103 ST
185617,R208,103 ST-CORONA
42146,R007,104 ST
68926,R354,104 ST
162357,R181,110 ST
42314,R008,111 ST
69131,R355,111 ST


Hmm... interesting.

Let's take another tack here.  Let's assume most things are "REGULAR".  Or... that'd be easy to check...

In [34]:
df.loc[:,'DESC'].value_counts()

REGULAR       200444
RECOVR AUD       753
Name: DESC, dtype: int64

Yeah... I'd call that mostly.

So, what are our widgets here?  Well, what are our timestamps?

In [55]:
print(len(df[df['DESC']=='REGULAR'].loc[:,['DATE','TIME']].drop_duplicates()))
df[df['DESC']=='REGULAR'].loc[:,['DATE','TIME']].drop_duplicates().head(10)

13924


,DATE,TIME
0,04/08/2017,00:00:00
1,04/08/2017,04:00:00
2,04/08/2017,08:00:00
3,04/08/2017,12:00:00
4,04/08/2017,16:00:00
5,04/08/2017,20:00:00
6,04/09/2017,00:00:00
7,04/09/2017,04:00:00
8,04/09/2017,08:00:00
9,04/09/2017,12:00:00


Nope.  Even "REGULAR" timestamps aren't perfect here.  But if we assume it's generally on a 4 hour pattern here.  This is one week, right?

In [57]:
df.loc[:,'DATE'].value_counts()

04/12/2017    31452
04/13/2017    28679
04/14/2017    28380
04/10/2017    28373
04/11/2017    28211
04/08/2017    28088
04/09/2017    28014
Name: DATE, dtype: int64

Yup.  And we can "see" it here.  We've got shy of 5000 things being recorded six times a day.

In [71]:
df.iloc[:,1:6].drop_duplicates().count()

UNIT        4691
SCP         4691
STATION     4691
LINENAME    4691
DIVISION    4691
dtype: int64

In [73]:
df.iloc[:,1:3].drop_duplicates().count()

UNIT    4691
SCP     4691
dtype: int64

I cannot escape the feeling that some of these columns are keys and some are descriptive.  Well... in any case, let's move on...

We can see even the "regular" timestamps aren't regular.  So we're going to need to do some work here.
Let's practice with one widget.

In [193]:
widget_key = df.iloc[:,1:3].drop_duplicates().iloc[0,:]
widget_key

UNIT        R051
SCP     02-00-00
Name: 0, dtype: object

In [119]:
widget = df.query("UNIT=='R469' and SCP=='00-05-01'")
len(widget)

42

Playing around I can see even the "REGULAR" timestamps that are "clean" don't all fall on the same four hour pattern.

Yeah... we're simply going to have to resample...

And for that, let's back up and parse the timestamp.

In [196]:
df2 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_170415.txt',
                       parse_dates = [['DATE','TIME']]
                      )

In [198]:
widget = df2.query("UNIT=='R051' and SCP=='02-00-00'").copy()
widget

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,ENTRIES,EXITS
0,2017-04-08 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6127245,2076158
1,2017-04-08 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6127285,2076165
2,2017-04-08 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6127304,2076196
3,2017-04-08 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6127409,2076284
4,2017-04-08 16:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6127654,2076350
5,2017-04-08 20:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6128002,2076398
6,2017-04-09 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6128172,2076417
7,2017-04-09 04:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6128213,2076419
8,2017-04-09 08:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6128226,2076440
9,2017-04-09 12:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6128293,2076512


In [199]:
widget.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 0 to 41
Data columns (total 10 columns):
DATE_TIME                                                               42 non-null datetime64[ns]
C/A                                                                     42 non-null object
UNIT                                                                    42 non-null object
SCP                                                                     42 non-null object
STATION                                                                 42 non-null object
LINENAME                                                                42 non-null object
DIVISION                                                                42 non-null object
DESC                                                                    42 non-null object
ENTRIES                                                                 42 non-null int64
EXITS                                                                  

In [200]:
widget.reset_index()
widget2 = widget.set_index(pd.DatetimeIndex(widget['DATE_TIME'],inplace=True))


In [201]:
widget2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 42 entries, 2017-04-08 00:00:00 to 2017-04-14 20:00:00
Data columns (total 10 columns):
DATE_TIME                                                               42 non-null datetime64[ns]
C/A                                                                     42 non-null object
UNIT                                                                    42 non-null object
SCP                                                                     42 non-null object
STATION                                                                 42 non-null object
LINENAME                                                                42 non-null object
DIVISION                                                                42 non-null object
DESC                                                                    42 non-null object
ENTRIES                                                                 42 non-null int64
EXITS                            

In [214]:
widget2.resample('1H').mean().fillna(method='ffill').diff(1)

,ENTRIES,EXITS
DATE_TIME,,
2017-04-08 00:00:00,NaN,NaN
2017-04-08 01:00:00,0.0,0.0
2017-04-08 02:00:00,0.0,0.0
2017-04-08 03:00:00,0.0,0.0
2017-04-08 04:00:00,40.0,7.0
2017-04-08 05:00:00,0.0,0.0
2017-04-08 06:00:00,0.0,0.0
2017-04-08 07:00:00,0.0,0.0
2017-04-08 08:00:00,19.0,31.0


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,00:00:00,REGULAR,6127245,2076158
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,04:00:00,REGULAR,6127285,2076165
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,08:00:00,REGULAR,6127304,2076196
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,12:00:00,REGULAR,6127409,2076284
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/08/2017,16:00:00,REGULAR,6127654,2076350
